<img src="./images/DLI_Header.png" style="width: 400px;">

# 1.0 Overview of the Class Environment

This notebook will introduce the basic knowledge of using AI clusters. You will have an overview of the Class Environment configured as an AI compute cluster. In addition, you will experiment with basic commands of the [SLURM cluster management](https://slurm.schedmd.com/overview.html).

### Learning Objectives

The goals of this notebook are to:
* Understand the hardware configuration available for the class
* Understand the basics commands for jobs submissions with SLURM
* Run simple test scripts allocating different GPU resources
* Connect interactively to a compute node and observe available resources

**[1.1 The Hardware Configuration Overview](#1.1-The-Hardware-Configuration-Overview)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[1.1.1 Check The Available CPUs](#1.1.1-Check-The-Available-CPUs)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[1.1.2 Check the Available GPUs](#1.1.2-Check-The-Available-GPUs)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[1.1.3 Check The Interconnect Topology](#1.1.3-Check-The-Interconnect-Topology)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[1.1.4 Bandwidth & Connectivity Tests](#1.1.4-Bandwidth-and-Connectivity-Tests)<br>
**[1.2 Basic SLURM Commands](#1.2-Basic-SLURM-Commands)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[1.2.1 Check the SLURM Configuration](#1.2.1-Check-the-SLURM-Configuration)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[1.2.2 Submit Jobs Using SRUN Command](#1.2.2-Submit-jobs-using-SRUN-Command)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[1.2.3 Submit Jobs Using SBATCH Command](#1.2.3-Submit-jobs-using-SBATCH-Command])<br>
&nbsp;&nbsp;&nbsp;&nbsp;[1.2.4 Exercise: Submit Jobs Using SBATCH Command Requesting More Resources](#1.2.4-Exercise-Submit-jobs-using-SBATCH-Command])<br>
**[1.3 Run Interactive Sessions](#1.3-Run-Interactive-Sessions)<br>**

---
# 1.1 The Hardware Configuration Overview


A modern AI cluster is a type of infrastructure designed for optimal Deep Learning model development. NVIDIA has designed DGXs servers as a full-stack solution for scalable AI development. Click the link to learn more about [DGX systems](https://www.nvidia.com/en-gb/data-center/dgx-systems/).

In this lab, in terms of GPU and networking hardware resources, each class environment is configured to access about half the resources of a DGX-1 server system (4 V100 GPUs, 4 NVlinks per GPU).

<img  src="images/nvlink_v2.png" width="600"/>

The hardware for this class has already been configured as a GPU cluster unit for Deep Learning. The cluster is organized as compute units (nodes) that can be allocated using a Cluster Manager (example SLURM). Among the hardware components, the cluster includes CPUs (Central Processing Units), GPUs (Graphics Processing Units), storage and networking.

Let's look at the GPUs, CPUs and network design available in this class.

## 1.1.1 Check The Available CPUs 

We can check the CPU information of the system using the `lscpu` command. 

This example of outputs shows that there are 16 CPU cores of the `x86_64` from Intel.
```
Architecture:                    x86_64
Core(s) per socket:              16
Model name:                      Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
```
For a complete description of the CPU processor architecture, check the `/proc/cpuinfo` file.


In [1]:
# Display information CPUs
! lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   48 bits physical, 48 bits virtual
CPU(s):                          96
On-line CPU(s) list:             0-95
Thread(s) per core:              1
Core(s) per socket:              48
Socket(s):                       2
NUMA node(s):                    4
Vendor ID:                       AuthenticAMD
CPU family:                      25
Model:                           1
Model name:                      AMD EPYC 7V13 64-Core Processor
Stepping:                        1
CPU MHz:                         2464.505
BogoMIPS:                        4890.88
Hypervisor vendor:               Microsoft
Virtualization type:             full
L1d cache:                       3 MiB
L1i cache:                       3 MiB
L2 cache:                        48 MiB
L3 cache:                        384 MiB
NUMA node0 CPU(s):               0-23
NUMA 

In [2]:
# Check the number of CPU cores
!grep 'cpu cores' /proc/cpuinfo | uniq

cpu cores	: 48


## 1.1.2 Check The Available  GPUs 

The NVIDIA System Management Interface `nvidia-smi` is a command for monitoring NVIDIA GPU devices. Several key details are listed such as the CUDA and  GPU driver versions, the number and type of GPUs available, the GPU memory each, running GPU process, etc.

In the following example, `nvidia-smi` command shows that there are 4 Tesla V100-SXM2 GPUs (ID 0-3), each with 16GB of memory. 

<img  src="images/nvidia_smi.png" width="600"/>

For more details, refer to the [nvidia-smi documentation](https://developer.download.nvidia.com/compute/DCGM/docs/nvidia-smi-367.38.pdf).

In [3]:
# Display information about GPUs
! nvidia-smi

Thu Mar 23 16:58:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Graphics Device     On   | 00000001:00:00.0 Off |                    0 |
| N/A   37C    P0    53W / 300W |      0MiB / 81251MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  Graphics Device     On   | 00000002:00:00.0 Off |                    0 |
| N/A   

## 1.1.3 Check The Available Interconnect Topology 



<img  align="right" src="images/nvlink_nvidia.png" width="420"/>

The multi-GPU system configuration needs a fast and scalable interconnect. [NVIDIA NVLink technology](https://www.nvidia.com/en-us/data-center/nvlink/) is a direct GPU-to-GPU interconnect providing high bandwidth and improving scalability for multi-GPU systems.

To check the available interconnect topology, we can use `nvidia-smi topo --matrix` command. In this class, we should get 4 NVLinks per GPU device. 

```
        GPU0    GPU1    GPU2    GPU3    CPU Affinity    NUMA Affinity
GPU0     X      NV1     NV1     NV2     0-31            N/A
GPU1    NV1      X      NV2     NV1     0-31            N/A
GPU2    NV1     NV2      X      NV2     0-31            N/A
GPU3    NV2     NV1     NV2      X      0-31            N/A

Where X= Self and NV# = Connection traversing a bonded set of # NVLinks
```

In this environment, notice only 1 link between GPU0 and GPU1, GPU2 while 2 links are shown between GPU0 and GPU3.

In [4]:
# Check Interconnect Topology 
! nvidia-smi topo --matrix

	GPU0	GPU1	GPU2	GPU3	CPU Affinity	NUMA Affinity
GPU0	 X 	NV12	SYS	SYS	0-23	0
GPU1	NV12	 X 	SYS	SYS	24-47	1
GPU2	SYS	SYS	 X 	NV12	48-71	2
GPU3	SYS	SYS	NV12	 X 	72-95	3

Legend:

  X    = Self
  SYS  = Connection traversing PCIe as well as the SMP interconnect between NUMA nodes (e.g., QPI/UPI)
  NODE = Connection traversing PCIe as well as the interconnect between PCIe Host Bridges within a NUMA node
  PHB  = Connection traversing PCIe as well as a PCIe Host Bridge (typically the CPU)
  PXB  = Connection traversing multiple PCIe bridges (without traversing the PCIe Host Bridge)
  PIX  = Connection traversing at most a single PCIe bridge
  NV#  = Connection traversing a bonded set of # NVLinks


It is also possible to check the NVLink status and bandwidth using `nvidia-smi nvlink --status` command. You should see similar outputs per device.
```
GPU 0: Tesla V100-SXM2-16GB 
	 Link 0: 25.781 GB/s
	 Link 1: 25.781 GB/s
	 Link 2: 25.781 GB/s
	 Link 3: 25.781 GB/s
```

In [5]:
# Check nvlink status
! nvidia-smi nvlink --status

GPU 0: Graphics Device (UUID: GPU-92bb16cd-3f7b-fff6-85ba-d59f3ee01fec)
	 Link 0: 25 GB/s
	 Link 1: 25 GB/s
	 Link 2: 25 GB/s
	 Link 3: 25 GB/s
	 Link 4: 25 GB/s
	 Link 5: 25 GB/s
	 Link 6: 25 GB/s
	 Link 7: 25 GB/s
	 Link 8: 25 GB/s
	 Link 9: 25 GB/s
	 Link 10: 25 GB/s
	 Link 11: 25 GB/s
GPU 1: Graphics Device (UUID: GPU-000d0a4b-3c11-527c-5b9e-e9000609784a)
	 Link 0: 25 GB/s
	 Link 1: 25 GB/s
	 Link 2: 25 GB/s
	 Link 3: 25 GB/s
	 Link 4: 25 GB/s
	 Link 5: 25 GB/s
	 Link 6: 25 GB/s
	 Link 7: 25 GB/s
	 Link 8: 25 GB/s
	 Link 9: 25 GB/s
	 Link 10: 25 GB/s
	 Link 11: 25 GB/s
GPU 2: Graphics Device (UUID: GPU-4aa50531-07c0-fe5d-efca-a1fb8357d663)
	 Link 0: 25 GB/s
	 Link 1: 25 GB/s
	 Link 2: 25 GB/s
	 Link 3: 25 GB/s
	 Link 4: 25 GB/s
	 Link 5: 25 GB/s
	 Link 6: 25 GB/s
	 Link 7: 25 GB/s
	 Link 8: 25 GB/s
	 Link 9: 25 GB/s
	 Link 10: 25 GB/s
	 Link 11: 25 GB/s
GPU 3: Graphics Device (UUID: GPU-659119cb-c367-4cbb-a547-d02fb3aba084)
	 Link 0: 25 GB/s
	 Link 1: 25 GB/s
	 Link 2: 25 GB/s
	 Li

## 1.1.4 Bandwidth & Connectivity Tests


NVIDIA provides an application **p2pBandwidthLatencyTest** that demonstrates CUDA Peer-To-Peer (P2P) data transfers between pairs of GPUs by computing bandwidth and latency while enabling and disabling NVLinks. This tool is part of the code samples for CUDA Developers [cuda-samples](https://github.com/NVIDIA/cuda-samples.git). 

Example outputs are shown below. Notice the Device to Device (D\D) bandwidth differences when enabling and disabling NVLinks (P2P).

```
Bidirectional P2P=Enabled Bandwidth Matrix (GB/s)
   D\D     0      1      2      3 
     0 780.27  48.49  48.49  96.89 
     1  48.49 777.94  96.91  48.49 
     2  48.49  96.85 779.30  96.90 
     3  96.89  48.49  96.89 779.11 

Bidirectional P2P=Disabled Bandwidth Matrix (GB/s)
   D\D     0      1      2      3 
     0 777.94   9.56  14.41  14.48 
     1   9.49 780.47  14.19  14.16 
     2  14.39  14.18 779.89   9.51 
     3  14.44  14.24   9.65 780.08
```


In [6]:
# Tests on GPU pairs using P2P and without P2P 
#`git clone --depth 1 --branch v11.2 https://github.com/NVIDIA/cuda-samples.git`
! /dli/cuda-samples/bin/x86_64/linux/release/p2pBandwidthLatencyTest

[P2P (Peer-to-Peer) GPU Bandwidth Latency Test]
Device: 0, Graphics Device, pciBusID: 0, pciDeviceID: 0, pciDomainID:1
Device: 1, Graphics Device, pciBusID: 0, pciDeviceID: 0, pciDomainID:2
Device: 2, Graphics Device, pciBusID: 0, pciDeviceID: 0, pciDomainID:3
Device: 3, Graphics Device, pciBusID: 0, pciDeviceID: 0, pciDomainID:4
Device=0 CAN Access Peer Device=1
Device=0 CANNOT Access Peer Device=2
Device=0 CANNOT Access Peer Device=3
Device=1 CAN Access Peer Device=0
Device=1 CANNOT Access Peer Device=2
Device=1 CANNOT Access Peer Device=3
Device=2 CANNOT Access Peer Device=0
Device=2 CANNOT Access Peer Device=1
Device=2 CAN Access Peer Device=3
Device=3 CANNOT Access Peer Device=0
Device=3 CANNOT Access Peer Device=1
Device=3 CAN Access Peer Device=2

***NOTE: In case a device doesn't have P2P access to other one, it falls back to normal memcopy procedure.
So you can see lesser Bandwidth (GB/s) and unstable Latency (us) in those cases.

P2P Connectivity Matrix
     D\D     0     1  

---
# 1.2 Basic SLURM Commands

Now that we've seen how GPUs can communicate with each other over NVLink, let's go over how the hardware resources can be organized into compute nodes. These nodes can be managed by Cluster Manager such as [*Slurm Workload Manager*](https://slurm.schedmd.com/), an open source cluster management and job scheduler system for large and small Linux clusters. 


For this lab, we have configured a SLURM manager where the 4 available GPUs are partitioned into 2 nodes: **slurmnode1** 
and **slurmnode2**, each with 2 GPUs. 

Next, let's see some basic SLURM commands. More SLURM commands can be found in the [SLURM official documentation](https://slurm.schedmd.com/).

<img src="images/cluster_overview.png" width="500"/>

## 1.2.1 Check the SLURM Configuration

We can check the available resources in the SLURM cluster by running `sinfo`. The output will show that there are 2 nodes in the cluster **slurmnode1** and **slurmnode2**. Both nodes are currently idle.

![title](images/slurm_config.png)

In [7]:
# Check available resources in the cluster
! sinfo

PARTITION AVAIL  TIMELIMIT  NODES  STATE NODELIST
slurmpar*    up   infinite      2   idle slurmnode[1-2]


##  1.2.2 Submit Jobs Using `srun` Command

The `srun` command allows to running parallel jobs. 

The argument **-N** (or *--nodes*) can be used to specify the nodes allocated to a job. It is also possible to allocate a subset of GPUs available within a node by specifying the argument **-G (or --gpus)**.

Check out the [SLURM official documentation](https://slurm.schedmd.com/) for more arguments.

To test running parallel jobs, let's submit a job that requests 1 node (2 GPUs) and run a simple command on it: `nvidia-smi`. We should see the output of 2 GPUs available in the allocated node.

In [8]:
# run nvidia-smi slurm job with 1 node allocation
! srun -N 1 nvidia-smi

Thu Mar 23 16:59:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Graphics Device     On   | 00000001:00:00.0 Off |                    0 |
| N/A   38C    P0    53W / 300W |      0MiB / 81251MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  Graphics Device     On   | 00000002:00:00.0 Off |                    0 |
| N/A   

Great! Let's now allocate 2 nodes and run again `nvidia-smi` command.

We should see the results of both nodes showing the available GPU devices. Notce that the stdout might be scrumbled due to the asynchronous and parallel execution of `nvidia-smi` command in the two nodes.

In [9]:
# run nvidia-smi slurm job with 2 node allocation.
! srun -N 2 nvidia-smi

Thu Mar 23 17:00:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
Thu Mar 23 17:00:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage |

## 1.2.3 Submit Jobs Using `sbatch` Command 

In the previous examples, we allocated resources to run one single command. For more complex jobs, the `sbatch` command allows submitting batch scripts to SLURM by specifying the resources and all environment variables required for executing the job. `sbatch` will transfer the execution to the SLURM Manager after automatically populating the arguments.

In the batch script below, `#SBATCH ...` is used to specify resources and other options relating to the job to be executed:

```
        #!/bin/bash
        #SBATCH -N 1                               # Node count to be allocated for the job
        #SBATCH --job-name=dli_firstSlurmJob       # Job name
        #SBATCH -o /dli/megatron/logs/%j.out       # Outputs log file 
        #SBATCH -e /dli/megatron/logs/%j.err       # Errors log file

        srun -l my_script.sh                       # my SLURM script 
```

Before we submit the `sbatch` batch script, let's first prepare a job that will be executed: a short batch script that will sleep for 2 seconds before running the `nvidia-smi` command.

In [ ]:
!chmod +x /dli/code/test.sh
# Check the batch script 
!cat /dli/code/test.sh

To submit this batch script job, let's create an `sbatch` script that initiates the resources to be allocated and submits the test.sh job.

The following cell will edit the `test_sbatch.sbatch` script allocating 1 node.

In [ ]:
%%writefile /dli/code/test_sbatch.sbatch
#!/bin/bash

#SBATCH -N 1
#SBATCH --job-name=dli_firstSlurmJob
#SBATCH -o /dli/megatron/logs/%j.out
#SBATCH -e /dli/megatron/logs/%j.err

srun -l /dli/code/test.sh  

In [ ]:
# Check the sbatch script 
! cat /dli/code/test_sbatch.sbatch

Now let's submit the `sbatch` job and check the SLURM scheduler. The batch script will be queued and executed when the requested resources are available.

The `squeue` command shows the running or pending jobs. An output example is shown below: 

```
Submitted batch job **
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                **  slurmpar test_sba    admin  R       0:01      1 slurmnode1

```

It shows the SLURM Job ID, Job's name, the user ID, Job's Status (R=running), running duration and the allocated node name.

The following cell submits the `sbatch` job, collects the `JOBID` variable (for querying later the logs) and checks the jobs in the SLURM scheduling queue.

In [ ]:
# Submit the job
! sbatch /dli/code/test_sbatch.sbatch

# Get the JOBID variable
JOBID=!squeue -u admin | grep dli | awk '{print $1}'
slurm_job_output='/dli/megatron/logs/'+JOBID[0]+'.out'

# check the jobs in the SLURM scheduling queue
! squeue

The output log file for the executed job (**JOBID.out**) is automatically created to gather the outputs.

In our case, we should see the results of `nvidia-smi` command that was executed in the `test.sh` script submitted with 1 node allocation. Let's have a look at execution logs:


In [ ]:
# Wait 3 seconds to let the job execute and get the populated logs 
! sleep 3

# Check the execution logs 
! cat $slurm_job_output

## 1.2.4  Exercise: Submit Jobs Using `sbatch` Command  Requesting More Resources


Using what you have learned, submit the previous `test.sh` batch script with the `sbatch` command on **2 nodes** allocation.

To do so, you will need to:
1. Modify the `test_sbatch.sbatch` script to allocate 2 Nodes 
2. Submit the script again using `sbatch` command
3. Check the execution logs 


If you get stuck, you can look at the [solution](solutions/ex1.2.4.ipynb).

In [ ]:
# 1. Modify the `test_sbatch.sbatch` script to allocate 2 Nodes 

In [ ]:
# 2. Submit the script again using `sbatch` command

In [ ]:
# 3. Check the execution logs 

---
# 1.3 Run Interactive Sessions 

Interactive sessions allow to connect directly to a worker node and interact with it through the terminal. 

The SLURM manager allows to allocate resources in interactive session using the `--pty` argument as follows: `srun -N 1 --pty /bin/bash`. 
The session is closed when you exit the node or you cancel the interactive session job using the command `scancel JOBID`.


Since this is an interactive session, first, we need to launch a terminal window and submit a slurm job allocating resources in interactive mode. To do so, we will need to follow the 3 steps: 
1. Launch a terminal session
2. Check the GPUs resources using the command `nvidia-smi` 
3. Run an interactive session requesting 1 node by executing `srun -N 1 --pty /bin/bash`
4. Check the GPUs resources using the command `nvidia-smi` again 

Let's run our first interactive job requesting 1 node and check what GPU resources are at our disposal. 

![title](images/interactive_launch.png)

Notice that while connected to the session, the host name as displayed in the command line changes from "lab" (login node name) to "slurmnode1" indicating that we are now successfully working on a remote worker node.

Run the following cell to get a link to open a terminal session and the instructions to run an interactive session.

In [ ]:
%%html

<pre>
   Step 1: Open a terminal session by following this <a href="", data-commandlinker-command="terminal:create-new">Terminal link</a>
   Step 2: Check the GPUs resources: <font color="green">nvidia-smi</font>
   Step 3: Run an interactive session: <font color="green">srun -N 1 --pty /bin/bash</font>
   Step 4: Check the GPUs resources again: <font color="green">nvidia-smi</font>
</pre>

---
<h2 style="color:green;">Congratulations!</h2>

You've made it through the first section of the course and are ready to begin training Deep Learning models on multiple GPUs. <br>

Before moving on, we need to make sure that no jobs are still running or waiting on the SLURM queue. 
Let's check the SLURM jobs queue by executing the following cell:

In [ ]:
# Check the SLURM jobs queue 
!squeue

If there are still jobs running or pending, execute the following cell to cancel all the user's jobs using the `scancel` command. 

In [ ]:
# Cancel admin user jobs
! scancel -u $USER

# Check again the SLURM jobs queue (should be either empty, or the status TS column should be CG)
! squeue

Next, we will be running basic GPT language model training on different distribution configurations. Move on to [02_GPT_LM_pretrainings.ipynb](02_GPT_LM_pretrainings.ipynb).